In [0]:
from pyspark.sql.functions import col, from_json, expr, row_number, lit
from pyspark.sql.window import Window

In [0]:
BRONZE_WEB_PATH = "class_dsrp_dtb.default.bronze_web"
BRONZE_APP_PATH = "class_dsrp_dtb.default.bronze_app"
SILVER_TRANSACCIONES_TABLE = "silver_transacciones"

In [0]:
spark.conf.set(
    "spark.sql.legacy.timeParserPolicy",
    "LEGACY"
)

Lectura de datos desde Azure SQL

In [0]:
df_bronze_web = spark.table(BRONZE_WEB_PATH)

Lectura de datos en streaming desde CosmosDB

In [0]:
df_bronze_app_cosmos  = spark.table(BRONZE_APP_PATH)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, LongType, DateType, TimestampType

In [0]:
bronze_app_schema = StructType([
    StructField("id_trx", IntegerType(), True),
    StructField("id_usuario", StringType(), True),
    StructField("operacion", StringType(), True),
    StructField("monto", DoubleType(), True),
    StructField("fecha", StringType(), True),
    StructField("hora", StringType(), True),
])

In [0]:
df_bronze_app = df_bronze_app_cosmos.select(from_json(col("document").cast("string"), bronze_app_schema).alias("c"), "*").select("c.*")

Consolidado de transacciones

In [0]:
%sql
DROP TABLE IF EXISTS class_dsrp_dtb.default.silver_transacciones;

CREATE TABLE IF NOT EXISTS class_dsrp_dtb.default.silver_transacciones (
  canal STRING,
  id_trx INT,
  id_usuario STRING,
  operacion STRING,
  monto DOUBLE,
  fecha DATE,
  hora STRING,
  fecha_hora TIMESTAMP
)
USING DELTA;

In [0]:
from pyspark.sql.functions import lit, to_date, cast, concat_ws, to_timestamp, concat_ws

In [0]:
df_bronze_web = df_bronze_web \
    .withColumn("canal", lit("web")) \
    .withColumn("fecha", to_date(col("fecha"), "yyyy-MM-dd")) \
    .withColumn("hora", col("hora").cast("string")) \
    .withColumn("fecha_hora", to_timestamp(concat_ws(" ", col("fecha"), col("hora")), "yyyy-MM-dd HH:mm:ss"))

df_bronze_web.write \
    .mode("append") \
    .saveAsTable("silver_transacciones")

In [0]:
df_bronze_app = df_bronze_app \
    .withColumn("canal", lit("app")) \
    .withColumn("fecha", to_date(col("fecha"), "yyyy-MM-dd")) \
    .withColumn("hora", col("hora").cast("string")) \
    .withColumn("fecha_hora", to_timestamp(concat_ws(" ", col("fecha"), col("hora")), "yyyy-MM-dd HH:mm:ss"))

df_bronze_app.write \
    .mode("append") \
    .saveAsTable("silver_transacciones")

Eliminar duplicados

In [0]:
silver_transacciones = spark.table(SILVER_TRANSACCIONES_TABLE)
display(silver_transacciones)

canal,id_trx,id_usuario,operacion,monto,fecha,hora,fecha_hora
web,501,3035,DEPOSITO,2985.45,2022-06-19,08:00:00.0,2022-06-19T08:00:00.000+0000
web,502,3028,VENTA,807.68,2023-11-02,08:00:00.0,2023-11-02T08:00:00.000+0000
web,503,3023,PAGO,1000.53,2020-08-01,08:00:00.0,2020-08-01T08:00:00.000+0000
web,504,3013,PAGO,2439.67,2024-08-12,08:00:00.0,2024-08-12T08:00:00.000+0000
web,505,3042,RETIRO,4651.65,2022-11-11,08:00:00.0,2022-11-11T08:00:00.000+0000
web,506,3011,COMPRA,3258.69,2025-04-26,08:00:00.0,2025-04-26T08:00:00.000+0000
web,507,3053,DEPOSITO,3460.41,2022-06-10,08:00:00.0,2022-06-10T08:00:00.000+0000
web,508,3074,PAGO,3402.76,2020-03-21,08:00:00.0,2020-03-21T08:00:00.000+0000
web,509,3069,COMPRA,1117.49,2024-05-15,08:00:00.0,2024-05-15T08:00:00.000+0000
web,510,3030,RETIRO,703.29,2023-12-03,08:00:00.0,2023-12-03T08:00:00.000+0000


In [0]:
df_sin_duplicados = silver_transacciones.dropDuplicates()
df_sin_duplicados.write \
    .mode("overwrite") \
    .saveAsTable("silver_transacciones")

In [0]:
silver_transacciones = spark.table(SILVER_TRANSACCIONES_TABLE)
display(silver_transacciones)

canal,id_trx,id_usuario,operacion,monto,fecha,hora,fecha_hora
web,558,3013,DEPOSITO,4076.78,2020-07-09,08:00:00.0,2020-07-09T08:00:00.000+0000
web,560,3044,PAGO,4972.54,2025-01-02,08:00:00.0,2025-01-02T08:00:00.000+0000
web,585,3026,VENTA,4202.55,2025-03-21,08:00:00.0,2025-03-21T08:00:00.000+0000
web,529,3010,VENTA,82.91,2023-01-04,08:00:00.0,2023-01-04T08:00:00.000+0000
web,523,3064,VENTA,3581.61,2023-01-29,08:00:00.0,2023-01-29T08:00:00.000+0000
web,512,3069,RETIRO,111.68,2021-08-05,08:00:00.0,2021-08-05T08:00:00.000+0000
web,538,3093,COMPRA,2099.29,2023-10-06,08:00:00.0,2023-10-06T08:00:00.000+0000
web,554,3056,COMPRA,929.01,2022-01-27,08:00:00.0,2022-01-27T08:00:00.000+0000
web,579,3095,PAGO,3741.41,2024-02-10,08:00:00.0,2024-02-10T08:00:00.000+0000
web,507,3053,DEPOSITO,3460.41,2022-06-10,08:00:00.0,2022-06-10T08:00:00.000+0000
